In [1]:
!pip install groq python-dotenv numpy tqdm datasets

In [2]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

In [3]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama-3.1-8b-instant"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [4]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello. What math problem would you like help with today?


#### GSM8K 데이터셋 확인해보기

In [5]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [6]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [7]:
### 수정해도 됩니다!
def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama-3.1-8b-instant",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)
            predicted_answer = extract_final_answer(response)

            if isinstance(predicted_answer, str):
                try:
                    predicted_answer = float(predicted_answer.replace(",", ""))
                except ValueError:
                    predicted_answer = None
                    
            if predicted_answer is not None:
                diff = abs(predicted_answer - correct_answer)
                is_correct = diff < 1e-5
            else:
                is_correct = False
            
            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [8]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [9]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate only the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [10]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!
shot_list = [0, 3, 5]
for shot in shot_list:
    print(f"{shot}-shot 테스트 시작")
    
    PROMPT = construct_direct_prompt(shot)
    VERBOSE = False

    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        VERBOSE=VERBOSE,
        num_samples=50
    )

    file_name = f"direct_prompting_{shot}.txt"
    save_final_result(results, accuracy, file_name)

0-shot 테스트 시작


 10%|█         | 5/50 [00:01<00:17,  2.52it/s]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:04<00:17,  2.35it/s]

Progress: [10/50]
Current Acc.: [60.00%]


...

Progress: [40/50]
Current Acc.: [75.00%]


 90%|█████████ | 45/50 [03:56<00:23,  4.73s/it]

Progress: [45/50]
Current Acc.: [73.33%]


100%|██████████| 50/50 [04:21<00:00,  5.24s/it]

Progress: [50/50]
Current Acc.: [74.00%]


### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [9]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    #TODO: 프롬프트를 작성해주세요!
    prompt=(
        "Instruction:\n"
        "Solve the following math problem by thinking through it step-by-step.\n"
        "First, describe the calculation process in the 'Scratchpad' section. "
        "Then, provide the final numerical value in the 'Answer' section.\n"
    )

    for i in range(num_examples):
        #TODO: CoT example을 만들어주세요!
        cur_question = train_dataset['question'][i]
        cur_process = train_dataset['answer'][i].split("####")[0].strip()
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Scratchpad:\n{cur_process}\n"
        prompt += f"Answer:{cur_answer}\n"
        pass

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [10]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!

shot_list = [0, 3, 5]
for shot in shot_list:
    print(f"{shot}-shot 테스트 시작")
    
    PROMPT = construct_CoT_prompt(shot)
    VERBOSE = False

    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        VERBOSE=VERBOSE,
        num_samples=50
    )

    file_name = f"CoT_prompting_{shot}.txt"
    save_final_result(results, accuracy, file_name)

0-shot 테스트 시작


 10%|█         | 5/50 [00:02<00:22,  2.00it/s]

Progress: [5/50]
Current Acc.: [100.00%]


 20%|██        | 10/50 [00:06<00:27,  1.46it/s]

Progress: [10/50]
Current Acc.: [80.00%]


...

Progress: [40/50]
Current Acc.: [82.50%]


 90%|█████████ | 45/50 [13:06<00:36,  7.30s/it]

Progress: [45/50]
Current Acc.: [82.22%]


100%|██████████| 50/50 [13:49<00:00, 16.59s/it]

Progress: [50/50]
Current Acc.: [84.00%]


### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [ ]:
### 자유롭게 수정해도 됩니다! 완전히 새로 함수를 만들어도 돼요.
def construct_my_prompt(num_examples: int = 3):
    train_dataset = gsm8k_train
    
    sampled_indices = random.sample(range(len(train_dataset['question'])), num_examples)

    # 2. [수정 전략] 복잡한 Step 1,2,3 지시를 제거하고 '논리적 완결성'만 강조
    # 지시문과 예시가 100% 일치하게 만들어 모델의 혼란을 없앱니다.
    prompt = (
        "Instruction:\n"
        "You are an expert mathematician. Solve the math problem step-by-step.\n"
        "Write down your logical reasoning in the 'Solution' section.\n"
        "Make sure to calculate carefully and double-check your answer.\n"
        "Finally, write the result after '####'.\n\n"
    )

    for idx in sampled_indices:
        cur_question = train_dataset['question'][idx]
        
        # 데이터 분리
        parts = train_dataset['answer'][idx].split("####")
        cur_solution = parts[0].strip()
        cur_answer = parts[-1].strip()

        # 예시 포맷: 깔끔하게 통일
        prompt += f"### Example\n"
        prompt += f"Question: {cur_question}\n"
        prompt += f"Solution:\n{cur_solution}\n"
        prompt += f"Answer: {cur_answer}\n\n"

    prompt += "### Your Task\n"
    prompt += "Question: {question}\n"
    prompt += "Answer:\nLet's think step by step.\nSolution:" 

    return prompt

In [12]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!
shot_list = [0, 3, 5]
for shot in shot_list:
    print(f"{shot}-shot 테스트 시작")
    
    PROMPT = construct_my_prompt(shot)
    VERBOSE = False

    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        VERBOSE=VERBOSE,
        num_samples=50
    )

    file_name = f"My_prompting_{shot}.txt"
    save_final_result(results, accuracy, file_name)

0-shot 테스트 시작


 10%|█         | 5/50 [00:03<00:37,  1.21it/s]

Progress: [5/50]
Current Acc.: [60.00%]


 20%|██        | 10/50 [00:07<00:29,  1.34it/s]

Progress: [10/50]
Current Acc.: [60.00%]


...

Progress: [40/50]
Current Acc.: [77.50%]


 90%|█████████ | 45/50 [08:47<00:54, 10.89s/it]

Progress: [45/50]
Current Acc.: [80.00%]


100%|██████████| 50/50 [09:47<00:00, 11.74s/it]

Progress: [50/50]
Current Acc.: [82.00%]


### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!